In [17]:
import json
from functools import reduce

In [2]:
json.loads("[[1,2],3]")

[[1, 2], 3]

In [3]:
test_explode = json.loads("[[[[[9,8],1],2],3],4]")

In [4]:
test_explode

[[[[[9, 8], 1], 2], 3], 4]

In [11]:
def explode(n, level=0):
    if not isinstance(n, int):
        l, r = n
        if level >= 4:
            return 0, True, l, r
        else:
            l, reduced, expl, expr = explode(l, level + 1)
            if reduced:
                if expr != 0:
                    r = add_left(r, expr)
                    expr = 0
            else:
                r, reduced, expl, expr = explode(r, level + 1)
                if reduced:
                    if expl != 0:
                        l = add_right(l, expl)
                        expl = 0
            if reduced:
                return (l, r), True, expl, expr
    return n, False, 0, 0

def add_left(n, m):
    if isinstance(n, int):
        return n + m
    else:
        a, b = n
        return add_left(a, m), b

def add_right(n, m):
    if isinstance(n, int):
        return n + m
    else:
        a, b = n
        return a, add_right(b, m)

def splat(n):
    if isinstance(n, int):
        if n > 9:
            a = n // 2
            return (a, n - a), True
    else:
        l, r = n
        l, reduced = splat(l)
        if not reduced:
            r, reduced = splat(r)
        if reduced:
            return (l, r), True
    return n, False

def mag(n):
    if isinstance(n, int):
        return n
    l, r = n
    return 3 * mag(l) + 2 * mag(r)

def fish_reduce(n):
    reduced = True
    while reduced:
        n, reduced, *_ = explode(n, 0)
        if not reduced:
            n, reduced = splat(n)
    return n

def add_fish(a, b):
    return fish_reduce((a, b))

In [9]:
explode(test_explode)

(((((0, 9), 2), 3), 4), True, 9, 0)

In [12]:
add_fish(json.loads("[[[[4,3],4],4],[7,[[8,4],9]]]"), (1, 1))

((((0, 7), 4), ((7, 8), (6, 0))), (8, 1))

In [13]:
def load_data(datafile):
    fish = []
    with open(datafile, 'r') as fp:
        for l in fp.readlines():
            fish.append(json.loads(l.strip()))
    return fish

In [15]:
load_data("test.txt")

[[[[4, 2], 2], 6], [8, 7]]

In [22]:
fish_numbers = load_data("test2.txt")
fish_numbers = map(fish_reduce, fish_numbers)
result = reduce(add_fish, fish_numbers)
result

((((6, 6), (7, 6)), ((7, 7), (7, 0))), (((7, 7), (7, 7)), ((7, 8), (9, 9))))

In [23]:
mag(result)

4140

In [26]:
fish_numbers = load_data("input.txt")
fish_numbers = map(fish_reduce, fish_numbers)
fish_list = list(fish_numbers)
max_mag = -1e6
for i, ni in enumerate(fish_list):
    for j, nj in enumerate(fish_list):
        if i == j:
            continue
        m = mag(add_fish(ni, nj))
        if m > max_mag:
            max_mag = m
max_mag

4563